# Using enterprise to analyze PTA data

### Developed by Steve Taylor, Subverted by Jeff Hazboun

Based on original worksheets in the IPTA GitHub [page](https:/github.com/ipta/gwa_tutorials).

In this notebook you will learn:
* How to use `enterprise` to interact with IPTA data,
* How to setup an analysis of indiviudual pulsar noise properties,
* How to post-process your results.

# Load modules

In [ ]:
% matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from __future__ import division

import numpy as np
import os, glob, json 
import matplotlib.pyplot as plt
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

# Single pulsar analysis

* `enterprise` is structured so that one first creates `parameters`, then `signals` that these `parameters` belong to, then finally a `model` that is the union of all `signals` and the `data`.

* We will show this explciitly below, then introduce some model shortcut code that will make your life easier.
* We test on the NANOGrav data for `J0613-0200`. We use NANOGrav data here to illustrate the use of `ECORR`, the white noise in pulsar timing data correlated amongst TOAs from the same observing run. 

## Load in `Pulsar` class

* The `enterprise` Pulsar class uses `libstempo` to read in `par` and `tim` files, then stores all pulsar data into a `Pulsar` object. This object contains all data and meta-data needed for the ensuing pulsar and PTA analysis. You no longer to reference the `par` and `tim` files after this cell.
* Note below that you can explicitly declare which version of the JPL solar-system ephemeris model that will be used to compute the Roemer delay between the geocenter and the barycenter (e.g. `DE436`). Otherwise the default values will be taken from the `par` files. Explicitly declaring the version here is good practice.
* You can also explicitly set the clock file to a version of `BIPM`, e.g. `BIPM(2015)`. This is less important, and you can let the code take the value from the `par` file.
* When you execute the following cell, you will get warnings like `WARNING: Could not find pulsar distance for PSR ...`. Don't worry! This is expected, and fine. Not all pulsars have well constrained distances, and will be set to `1 kpc` with a `20%` uncertainty.

In [ ]:
par = './J0613-0200_NANOGrav_11yv0.gls.par'
tim = './J0613-0200_NANOGrav_11yv0.tim'
psr = Pulsar(par, tim, ephem='DE436', clk=None)

In [ ]:
# find the maximum time span to set red-noise/DM-variation frequency sampling
tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = np.max(tmax) - np.min(tmin)

In [ ]:
# define selection by observing backend
selection = selections.Selection(selections.by_backend)

# special selection for ECORR only use wideband NANOGrav data
selection2 = selections.Selection(selections.nanograv_backends)

## Create parameters

In [ ]:
# white noise parameters
efac = parameter.Uniform(0.01, 10.0)
equad = parameter.Uniform(-8.5, -5)
ecorr = parameter.Uniform(-8.5, -5)
# efac = parameter.Constant() 
# equad = parameter.Constant() 
# ecorr = parameter.Constant() 

# red noise parameters
log10_A = parameter.Uniform(-20, -11)
gamma = parameter.Uniform(0, 7)

# dm-variation parameters
log10_A_dm = parameter.Uniform(-20, -11)
gamma_dm = parameter.Uniform(0, 7)

## Create signals

In [ ]:
# white noise
ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection2)

# red noise (powerlaw with 30 frequencies)
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)

# DM-variations (powerlaw with 30 frequencies)
dm_basis = utils.createfourierdesignmatrix_dm(nmodes=30, Tspan=Tspan)
dm_pl = utils.powerlaw(log10_A=log10_A_dm, gamma=gamma_dm)
dm_gp = gp_signals.BasisGP(dm_pl, dm_basis, name='dm_gp')

# timing model
tm = gp_signals.TimingModel(use_svd=True)

## Piece the full model together

In [ ]:
# intialize a single-pulsar pta model
models = ef + eq + ec + rn + dm_gp + tm
    
pta = signal_base.PTA(models(psr))

In [ ]:
pta.params

## Draw initial sample from model parameter space

In [ ]:
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

## Setup sampler (simple, with no tricks)

In [ ]:
# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2) # helps to tune MCMC proposal distribution

# where chains will be written to
outdir = './chains/noise_run_{}/'.format(str(psr.name))

# sampler object
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov,
                 outDir=outdir, 
                 resume=False)

## Sample the parameter space

In [ ]:
# sampler for N steps
N = int(1e6)

# SCAM = Single Component Adaptive Metropolis
# AM = Adaptive Metropolis
# DE = Differential Evolution
## You can keep all these set at default values
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

## Simple post-processing

In [ ]:
chain = np.loadtxt(outdir + 'chain_1.txt')
burn = int(0.25 * chain.shape[0])

In [ ]:
# Find column of chain file corresponding to a parameter
ind = list(pta.param_names).index('J0613-0200_Rcvr_800_GUPPI_efac')#('J0613-0200_log10_A')

In [ ]:
# Make trace-plot to diagnose sampling
plt.plot(chain[burn:, ind])
plt.show()

In [ ]:
# Plot a histogram of the marginalized posterior distribution
plt.hist(chain[burn:,ind], 50, normed=True, histtype='stepfilled', 
         lw=2, color='C0', alpha=0.5);
plt.xlabel('J0613-0200_log10_A')
plt.ylabel('PDF')
plt.show()

In [ ]:
# Make 2d histogram plot
ind_redA = list(pta.param_names).index('J0613-0200_log10_A')
ind_redgam = list(pta.param_names).index('J0613-0200_gamma')
fig = corner.corner(chain[burn:, [ind_redA, ind_redgam]], 
                    labels=['J0613-0200_log10_A', 'J0613-0200_gamma'],
                   levels=[0.68,0.95]);

__NOTE__: I am leaving the activities below in the worksheet so you see how much more easily this can be done. It's probably not worth pulling this repo right now. At this point your time is actually better spent going to the `stochastic_gwb_analysis.ipynb` to see how to do an analysis for a gravitational wave background. 

## Now, the easy way to do all of this

Many of us have created shortcuts to carry out these tasks. You will find them in `enterprise_extensions`: https://github.com/stevertaylor/enterprise_extensions. Clone this repo, go into the cloned repo directory, then execute `pip install .`

This will install the package on your local machine.

In [ ]:
import enterprise_extensions
from enterprise_extensions import models, model_utils

In [ ]:
# Create a single pulsar model
pta = models.model_singlepsr_noise(psr, psd='powerlaw', 
                                   noisedict=params, 
                                   white_vary=False,
                                   dm_var=True, 
                                   dm_psd='powerlaw', dm_annual=True)

In [ ]:
# Setup a sampler instance.
# This will add some fanicer stuff than before, like prior draws, 
# and custom sample groupings.
sampler = model_utils.setup_sampler(pta, outdir=outdir, resume=False)

In [ ]:
# sampler for N steps
N = int(1e6)
x0 = x0 = np.hstack(p.sample() for p in pta.params)

# SCAM = Single Component Adaptive Metropolis
# AM = Adaptive Metropolis
# DE = Differential Evolution
## You can keep all these set at default values
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

In [ ]:
chain = np.loadtxt(outdir + '/chain_1.txt')
burn = int(0.25*chain.shape[0])
pars = np.loadtxt(outdir + '/pars.txt', dtype=np.unicode_)

pp = model_utils.PostProcessing(chain, pars)

In [ ]:
pp.plot_trace()